Install

In [1]:
#test
#(studiolab) studio-lab-user@default:~/sagemaker-studiolab-notebooks/AI-OT24$ tensorboard --logdir logs/
#!pip install -q ib_insync numpy pandas yfinance matplotlib ta seaborn gymnasium stable-baselines3 sb3-contrib tensorflow scikit-learn quantstats imitation fredapi typing


Install requirements and import all functions

In [1]:
import os
os.chdir("/home/studio-lab-user/sagemaker-studiolab-notebooks/AI-OT24/Reinforcement-Learning-Stock-Porfolio-Managment")
!pip install -q -r requirements.txt

In [2]:
import sys
sys.path.append('./utils')
import trading_functions  # Import your library
import inspect  # Used to inspect the module

# Get all functions from the library
all_functions = [func for func, obj in inspect.getmembers(trading_functions) if inspect.isfunction(obj)]

# Print the function names
print("Functions in the 'trading_functions' library:")
for function in all_functions:
    print(function)

2024-11-19 18:26:37.710684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Functions in the 'trading_functions' library:
HER
accuracy_score
bin_reward_func
calculate_accuracy
check_env
clear_output
collect_expert_data
create_env
create_env_unique
create_evaluation_env
create_training_env
evaluate_all
evaluate_best
evaluate_buy
evaluate_model
evaluate_policy
evaluate_various
is_significant_reward_improvement
linear_schedule
lnr_reward_func
make_vec_env
save_model
smp_reward_func
sqh_reward_func
sqs_reward_func
stp_reward_func
train_model
train_test_split


In [3]:
import sys
sys.path.append('./utils')
from trading_functions import *
#from trading_functions import linear_schedule, TradingEnvUnique, TradingEnvUniqueMultiple, create_env, create_env_unique, evaluate_best, evaluate_buy, evaluate_model, evaluate_various, evaluate_all

Import

In [4]:
# Core Libraries
import os
import sys
import time
import datetime
import warnings
#warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all messages, 1 = info, 2 = warnings, 3 = errors

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

# Financial Data
import yfinance as yf
import quantstats as qs
import ta

# Machine Learning - Supervised Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

# Machine Learning - Deep Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.losses import BinaryCrossentropy

# Reinforcement Learning and Environments
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import A2C, DDPG, DQN, HER, PPO, SAC, TD3
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import (
    EvalCallback, StopTrainingOnRewardThreshold, StopTrainingOnNoModelImprovement
)
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecCheckNan
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib import ARS, MaskablePPO, RecurrentPPO, QRDQN, TRPO

# Imitation Learning
from imitation.algorithms import bc
from imitation.testing.reward_improvement import is_significant_reward_improvement
from imitation.data.types import Transitions

# Interactive Brokers API
from ib_insync import *

from typing import Callable

from collections import Counter

clear_output()


API Fred

In [5]:
from fredapi import Fred
fred = Fred(api_key='xxxxxxxxxxxxxxx')

RL

In [ ]:
# Define constants
SEED = 1
history_length = 5
reward_type = 'SQS'
stocks = ['AAPL', 'AMZN', 'META', 'MSFT', 'NVDA', 'TSLA']
n_envs = 10
n_steps = 8
total_timesteps = 10_000
batch_size = 16
learning_rate = 0.001
ent_coef = 0.10
log_interval = 1_000
eval_freq=1_000

current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_save_dir = f'./models/model_{current_date}'
log_dir = f'./logs/log_{current_date}'

# Training environment
env, vec_env = create_training_env(history_length, reward_type, '2020-01-01', '2023-06-30', stocks, n_envs)

# Validation environment
valid_env, vec_valid_env = create_evaluation_env(history_length, reward_type, '2023-07-01', '2023-12-30', stocks)

# Test environment
test_env, _ = create_evaluation_env(history_length, reward_type, '2024-01-01', '2024-10-30', stocks, n_envs=1)

# Evaluation callback for saving the best model
eval_callback = EvalCallback(
    vec_valid_env,
    n_eval_episodes=1,
    eval_freq=eval_freq,
    deterministic=True,
    verbose=1,
    best_model_save_path=model_save_dir,
)
stop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=3, min_evals=5, verbose=1)

# Train the model
model = train_model(
    model_name="PPO",  # Can be 'PPO', 'A2C', or 'DQN'
    create_model=True,
    vec_env=vec_env,
    iterations=1,
    train_timesteps=total_timesteps,
    log_frec=log_interval,
    log_base_dir=log_dir,
    n_steps=n_steps,
    batch_size=batch_size,
    learning_rate=learning_rate,
    ent_coef=ent_coef
)

# Evaluate the model
evaluate_model(env, model, 'Train', SEED, has_policy=False)
evaluate_model(valid_env, model, 'Valid', SEED, has_policy=False)
evaluate_model(test_env, model, 'Test', SEED, has_policy=False)

# Save the model to the Tests folder
save_model(model, model_save_dir)

# Close environments
env.close()
valid_env.close()
test_env.close()
vec_env.close()
vec_valid_env.close()

BC

In [5]:
# Define constants
SEED = 1
history_length = 5
reward_type = 'LNR'
stocks = ['AAPL', 'AMZN', 'META', 'MSFT', 'NVDA', 'TSLA']
n_envs = 1
n_steps = 8
total_timesteps = 10_000
batch_size = 64
learning_rate = 0.001
ent_coef = 0.10
log_interval = 10_000
eval_freq=1_000
log_dir = './logs'
model_save_dir = './models'

# Training environment
env, vec_env = create_training_env(history_length, reward_type, '2020-01-01', '2023-06-30', stocks, n_envs)
valid_env, vec_valid_env = create_training_env(history_length, reward_type, '2023-07-01', '2023-08-30', stocks, 1)
transitions, expert_actions = collect_expert_data(env, seed=SEED)
bc_trainer = train_model(
    model_name="BC",
    create_model=True,
    env=env,
    transitions=transitions,
    bc_batches=2000,
    bc_log_interval=1_000,
    batch_size=118,
    seed=1
)
model_actions, returns, total_reward = evaluate_model(env, name="BC",has_policy=True,SEED=SEED,trainer=bc_trainer)
action_counts = pd.Series(model_actions).value_counts()
print("Action Counts:")
print(action_counts)
model_actions, returns, total_reward = evaluate_model(valid_env, name="BC",has_policy=True,SEED=SEED,trainer=bc_trainer)
action_counts = pd.Series(model_actions).value_counts()
print("Action Counts:")
print(action_counts)
best_actions, best_returns, total_reward = evaluate_best(env, expert_actions, SEED)
action_counts = pd.Series(best_actions).value_counts()
print("Action Counts:")
print(action_counts)

# Close environments
env.close()
valid_env.close()
vec_env.close()
vec_valid_env.close()

/opt/conda/envs/sagemaker-distribution/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.df_unscaled to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.df_unscaled` for environment variables or `env.get_wrapper_attr('df_unscaled')` that will search the reminding wrappers.
  logger.warn(
/opt/conda/envs/sagemaker-distribution/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.steps to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.steps` for environment variables or `env.get_wrapper_attr('steps')` that will search the reminding wrappers.
  logger.warn(
/opt/conda/envs/sagemaker-distribution/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.history_length to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do

TypeError: Monitor.reset() takes 1 positional argument but 2 were given